# Create a new sqlalchemy Datasource
Use this notebook to configure a new sqlalchemy Datasource and add it to your project.

In [91]:
import great_expectations as gx
from great_expectations.cli.datasource import sanitize_yaml_and_save_datasource, check_if_datasource_name_exists
context = gx.get_context()

2023-02-22T20:30:00+0530 - INFO - FileDataContext loading zep config
2023-02-22T20:30:00+0530 - INFO - GxConfig.parse_yaml() failed with errors - [{'loc': ('xdatasources',), 'msg': 'field required', 'type': 'value_error.missing'}]
2023-02-22T20:30:00+0530 - INFO - GxConfig.parse_yaml() returning empty `xdatasources`
2023-02-22T20:30:00+0530 - INFO - Loading 'datasources' ->
{}
2023-02-22T20:30:00+0530 - INFO - Loaded 'datasources' ->
{}


## Customize Your Datasource Configuration

**If you are new to Great Expectations Datasources,** you should check out our [how-to documentation](https://docs.greatexpectations.io/docs/guides/connecting_to_your_data/connect_to_data_overview)

**My configuration is not so simple - are there more advanced options?**
Glad you asked! Datasources are versatile. Please see our [How To Guides](https://docs.greatexpectations.io/docs/guides/connecting_to_your_data/connect_to_data_overview)!

Give your datasource a unique name:

In [92]:
datasource_name = "my_datasource_new"

### For SQL based Datasources:

Here we are creating an example configuration based on the database backend you specified in the CLI.  The configuration contains an **InferredAssetSqlDataConnector**, which will add a Data Asset for each table in the database, a **ConfiguredAssetDataConnector**, which will add explicitly defined Data Assets, and a **RuntimeDataConnector**, which can accept SQL queries.

If any of these configuration options are not applicable, they can be removed. This is just an example, and you may customize this as you wish!

Also, if you would like to learn more about the **DataConnectors** used in this configuration, please see our docs on [InferredAssetDataConnectors](https://docs.greatexpectations.io/docs/guides/connecting_to_your_data/how_to_configure_an_inferredassetdataconnector), [ConfiguredAssetDataConnectors](https://docs.greatexpectations.io/docs/guides/connecting_to_your_data/how_to_configure_a_configuredassetdataconnector), and [RuntimeDataConnectors](https://docs.greatexpectations.io/docs/guides/connecting_to_your_data/how_to_configure_a_runtimedataconnector).

Credentials will not be saved until you run the last cell. The credentials will be saved in `uncommitted/config_variables.yml` which should not be added to source control.

In [93]:
host = "ab12092.ap-southeast-1"  # The account name (include region -- ex 'ABCD.us-east-1')
username = "SupriyaNeela"
database = "assignment_db"  # The database name
schema_name = "my_schema"  # The schema name
warehouse = "assignment_wh"  # The warehouse name
role = "admin"  # The role name
table_name = "employees"  # A table that you would like to add initially as a Data Asset
password = "Supriya@08"

In [94]:
example_yaml = f"""
name: {datasource_name}
class_name: Datasource
execution_engine:
  class_name: SqlAlchemyExecutionEngine
  credentials:
    host: {host}
    username: {username}
    database: {database}
    query:
      schema: {schema_name}
      warehouse: {warehouse}
      role: {role}
    password: {password}
    drivername: snowflake
data_connectors:
  default_runtime_data_connector_name:
    class_name: RuntimeDataConnector
    batch_identifiers:
      - default_identifier_name
  default_inferred_data_connector_name:
    class_name: InferredAssetSqlDataConnector
    include_schema_name: True
    introspection_directives:
      schema_name: {schema_name}
  default_configured_data_connector_name:
    class_name: ConfiguredAssetSqlDataConnector
    assets:
      {table_name}:
        class_name: Asset
        schema_name: {schema_name}
"""
print(example_yaml)


name: my_datasource_new
class_name: Datasource
execution_engine:
  class_name: SqlAlchemyExecutionEngine
  credentials:
    host: ab12092.ap-southeast-1
    username: SupriyaNeela
    database: assignment_db
    query:
      schema: my_schema
      warehouse: assignment_wh
      role: admin
    password: Supriya@08
    drivername: snowflake
data_connectors:
  default_runtime_data_connector_name:
    class_name: RuntimeDataConnector
    batch_identifiers:
      - default_identifier_name
  default_inferred_data_connector_name:
    class_name: InferredAssetSqlDataConnector
    include_schema_name: True
    introspection_directives:
      schema_name: my_schema
  default_configured_data_connector_name:
    class_name: ConfiguredAssetSqlDataConnector
    assets:
      employees:
        class_name: Asset
        schema_name: my_schema



# Test Your Datasource Configuration
Here we will test your Datasource configuration to make sure it is valid.

This `test_yaml_config()` function is meant to enable fast dev loops. **If your
configuration is correct, this cell will show you some snippets of the data
assets in the data source.** You can continually edit your Datasource config
yaml and re-run the cell to check until the new config is valid.

If you instead wish to use python instead of yaml to configure your Datasource,
you can use `context.add_datasource()` and specify all the required parameters.

In [95]:
context.test_yaml_config(yaml_config=example_yaml)

Attempting to instantiate class from config...
	Instantiating as a Datasource, since class_name is Datasource
	Successfully instantiated Datasource


ExecutionEngine class name: SqlAlchemyExecutionEngine
Data Connectors:
	default_configured_data_connector_name : ConfiguredAssetSqlDataConnector

	Available data_asset_names (1 of 1):
		employees (1 of 1): [{}]

	Unmatched data_references (0 of 0):[]

	default_inferred_data_connector_name : InferredAssetSqlDataConnector

	Available data_asset_names (2 of 2):
		my_schema.employees (1 of 1): [{}]
		my_schema.employeesvariant (1 of 1): [{}]

	Unmatched data_references (0 of 0):[]

	default_runtime_data_connector_name:RuntimeDataConnector

	Available data_asset_names (0 of 0):
		Note : RuntimeDataConnector will not have data_asset_names until they are passed in through RuntimeBatchRequest

	Unmatched data_references (0 of 0): []



## Save Your Datasource Configuration
Here we will save your Datasource in your Data Context once you are satisfied with the configuration. Note that `overwrite_existing` defaults to False, but you may change it to True if you wish to overwrite. Please note that if you wish to include comments you must add them directly to your `great_expectations.yml`.

In [97]:
sanitize_yaml_and_save_datasource(context, example_yaml, overwrite_existing=True)
context.list_datasources()

[{'data_connectors': {'default_inferred_data_connector_name': {'base_directory': '../data',
    'class_name': 'InferredAssetFilesystemDataConnector',
    'default_regex': {'group_names': ['data_asset_name'], 'pattern': '(.*)'},
    'module_name': 'great_expectations.datasource.data_connector'},
   'default_runtime_data_connector_name': {'class_name': 'RuntimeDataConnector',
    'module_name': 'great_expectations.datasource.data_connector',
    'assets': {'my_runtime_asset_name': {'batch_identifiers': ['runtime_batch_identifier_name'],
      'class_name': 'Asset',
      'module_name': 'great_expectations.datasource.data_connector.asset'}}}},
  'class_name': 'Datasource',
  'name': 'getting_started_datasource_test',
  'module_name': 'great_expectations.datasource',
  'execution_engine': {'class_name': 'PandasExecutionEngine',
   'module_name': 'great_expectations.execution_engine'}},
 {'data_connectors': {'default_runtime_data_connector_name': {'batch_identifiers': ['default_identifier_n

Now you can close this notebook and delete it!

In [78]:
print(context)

{
  "anonymous_usage_statistics": {
    "usage_statistics_url": "https://stats.greatexpectations.io/great_expectations/v1/usage_statistics",
    "data_context_id": "2cd4d811-37fe-43f7-ac78-f6564538194d",
    "explicit_url": false,
    "enabled": true,
    "explicit_id": true
  },
  "checkpoint_store_name": "checkpoint_store",
  "config_variables_file_path": "uncommitted/config_variables.yml",
  "config_version": 3.0,
  "data_docs_sites": {
    "local_site": {
      "class_name": "SiteBuilder",
      "show_how_to_buttons": true,
      "store_backend": {
        "class_name": "TupleFilesystemStoreBackend",
        "base_directory": "uncommitted/data_docs/local_site/"
      },
      "site_index_builder": {
        "class_name": "DefaultSiteIndexBuilder"
      }
    }
  },
  "datasources": {
    "getting_started_datasource_test": {
      "execution_engine": {
        "module_name": "great_expectations.execution_engine",
        "class_name": "PandasExecutionEngine"
      },
      "name": "

In [79]:
context.add_datasource(**example_yaml)

TypeError: great_expectations.data_context.data_context.abstract_data_context.AbstractDataContext.add_datasource() argument after ** must be a mapping, not str

In [80]:
import datetime
import logging

from great_expectations.core.batch import BatchRequest, RuntimeBatchRequest
from great_expectations.data_context import BaseDataContext
from great_expectations.data_context.types.base import (
    DataContextConfig,
    FilesystemStoreBackendDefaults,
)
from great_expectations.render.renderer import *
from great_expectations.render.view import DefaultJinjaPageView
from ruamel import yaml

In [81]:
# Second test for BatchRequest naming a table
batch_request = BatchRequest(
    datasource_name="my_datasource_new",
    data_connector_name="default_inferred_data_connector_name",
    #data_asset_name=f"{sfSchema.lower()}.employees",  # this is the name of the table you want to retrieve
    data_asset_name="my_schema.employees",
)
context.create_expectation_suite(
    expectation_suite_name="test_suite", overwrite_existing=True
)
validator = context.get_validator(
    batch_request=batch_request, expectation_suite_name="test_suite"
)

In [82]:
print(batch_request)

{
  "datasource_name": "my_datasource_new",
  "data_connector_name": "default_inferred_data_connector_name",
  "data_asset_name": "my_schema.employees"
}


In [83]:
print(validator.head())

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

   employee_id first_name  last_name      email phone_number  hire_date  \
0        198.0     Donald   OConnell  *********    *********  21-JUN-07   
1        199.0    Douglas      Grant  *********    *********  13-JAN-08   
2        200.0   Jennifer     Whalen  *********    *********  17-SEP-03   
3        201.0    Michael  Hartstein  *********    *********  17-FEB-04   
4        202.0        Pat        Fay  *********    *********  17-AUG-05   

     job_id   salary  manager_id  department_id                  elt_ts  \
0  SH_CLERK   2600.0       124.0           50.0 2023-02-06 04:21:14.674   
1  SH_CLERK   2600.0       124.0           50.0 2023-02-06 04:21:14.674   
2   AD_ASST   4400.0       101.0           10.0 2023-02-06 04:21:14.674   
3    MK_MAN  13000.0       100.0           20.0 2023-02-06 04:21:14.674   
4    MK_REP   6000.0       201.0           20.0 2023-02-06 04:21:14.674   

  elt_by      file_name  
0  local  employees.csv  
1  local  employees.csv  
2  local  employees.

In [89]:
df_test = validator.head().to_csv()
print(df_test)

Calculating Metrics:   0%|          | 0/1 [00:00<?, ?it/s]

,employee_id,first_name,last_name,email,phone_number,hire_date,job_id,salary,manager_id,department_id,elt_ts,elt_by,file_name
0,198.0,Donald,OConnell,*********,*********,21-JUN-07,SH_CLERK,2600.0,124.0,50.0,2023-02-06 04:21:14.674,local,employees.csv
1,199.0,Douglas,Grant,*********,*********,13-JAN-08,SH_CLERK,2600.0,124.0,50.0,2023-02-06 04:21:14.674,local,employees.csv
2,200.0,Jennifer,Whalen,*********,*********,17-SEP-03,AD_ASST,4400.0,101.0,10.0,2023-02-06 04:21:14.674,local,employees.csv
3,201.0,Michael,Hartstein,*********,*********,17-FEB-04,MK_MAN,13000.0,100.0,20.0,2023-02-06 04:21:14.674,local,employees.csv
4,202.0,Pat,Fay,*********,*********,17-AUG-05,MK_REP,6000.0,201.0,20.0,2023-02-06 04:21:14.674,local,employees.csv



In [49]:
validator.expect_column_values_to_not_be_null(column="employee_id")

Calculating Metrics:   0%|          | 0/8 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 49,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": []
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [50]:
validator.expect_column_min_to_be_between(column="salary", strict_min=0)

Calculating Metrics:   0%|          | 0/6 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "observed_value": 2100
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [66]:
# validator.expect_column_values_to_match_strftime_format(column="hire_date",strftime_format="%y-%b-%d")

In [67]:
validator.expect_column_values_to_be_between(column="elt_ts",min_value="2023-02-05 04:21:14.674",max_value="2023-02-06 04:21:14.674")

Calculating Metrics:   0%|          | 0/11 [00:00<?, ?it/s]

{
  "success": true,
  "result": {
    "element_count": 49,
    "unexpected_count": 0,
    "unexpected_percent": 0.0,
    "partial_unexpected_list": [],
    "missing_count": 0,
    "missing_percent": 0.0,
    "unexpected_percent_total": 0.0,
    "unexpected_percent_nonmissing": 0.0
  },
  "meta": {},
  "exception_info": {
    "raised_exception": false,
    "exception_traceback": null,
    "exception_message": null
  }
}

In [ ]:
# validator_futures.expect_column_values_to_not_be_null(column=column)
# validator_futures.expect_column_min_to_be_between(column="settle_price", strict_min=0)
# validator_futures.expect_column_pair_values_a_to_be_greater_than_b(
#     column_A="expiration_date", column_B="trade_date", or_equal=True
# )
# validator_futures.expect_column_stdev_to_be_between(
#     column="settle_price", min_value=1, max_value=5
# )
# validator_futures.save_expectation_suite(discard_failed_expectations=False)

In [21]:
# batch_kwargs = {
#     "datasource": "my_datasource_new",
#     "schema": "my_schema",  # schema is optional; default schema will be used if it is omitted
#     "table": "employees"  # note that the "table" key is used even to validate a view
# }

In [22]:
# expectation_suite_name = "npi_test.warning" 

In [23]:
# # Note, you can add the "overwrite_existing" flag to the below command if the suite
# # exists but you would like to replace it.
# context.create_expectation_suite(expectation_suite_name)

{
  "ge_cloud_id": null,
  "data_asset_type": null,
  "expectation_suite_name": "npi_test.warning",
  "meta": {
    "great_expectations_version": "0.15.48"
  },
  "expectations": []
}

In [26]:
from datetime import datetime
import great_expectations as gx
import great_expectations.jupyter_ux
from great_expectations.data_context.types.resource_identifiers import ValidationResultIdentifier

2023-02-20T17:49:02+0530 - INFO - Great Expectations logging enabled at 20 level by JupyterUX module.


In [31]:
# context = gx.data_context.DataContext()

2023-02-20T17:52:44+0530 - INFO - FileDataContext loading zep config
2023-02-20T17:52:44+0530 - INFO - GxConfig.parse_yaml() failed with errors - [{'loc': ('xdatasources',), 'msg': 'field required', 'type': 'value_error.missing'}]
2023-02-20T17:52:44+0530 - INFO - GxConfig.parse_yaml() returning empty `xdatasources`
2023-02-20T17:52:44+0530 - INFO - Loading 'datasources' ->
{}
2023-02-20T17:52:44+0530 - INFO - Loaded 'datasources' ->
{}


In [36]:
# batch = context.get_batch(batch_kwargs,expectation_suite_name)
# batch.head()

AttributeError: 'Datasource' object has no attribute 'get_batch'

In [33]:
print(batch_kwargs)

{'datasource': 'my_datasource_new', 'schema': 'my_schema', 'table': 'employees'}
